In [2]:
import os,os.path as osp
import io,gzip
import functools as ft,itertools as it
import pycurl
import numpy as np,pandas as pd

In [3]:
parent_dir = osp.abspath(osp.join(os.getcwd(),os.pardir))
cfg = osp.join(parent_dir,'genelinks.cfg')
osp.exists(cfg)

True

In [48]:
chrmsm = ['1','2','3','4','5','6','7','8','9','10','11','12','13',
          '14','15','16','17','18','19','20','21','22','X','Y','MT']
dir_p1 = 'chrom-{:02d}'
dir_p2 = 'chrom-{:s}'
for x in chrmsm:
    if x.isdecimal():
        dstr = dir_p1.format(int(x))
    else:
        dstr = dir_p2.format(x)
    dirs = osp.join(parent_dir,dstr)
    if not osp.exists(dirs):
        os.makedirs(dirs)
        print("created dir for %s"%x)
    else:
        print("found dir for %s"%x)

created dir for 1
created dir for 2
created dir for 3
created dir for 4
created dir for 5
created dir for 6
created dir for 7
created dir for 8
created dir for 9
created dir for 10
created dir for 11
created dir for 12
created dir for 13
created dir for 14
created dir for 15
created dir for 16
created dir for 17
created dir for 18
created dir for 19
created dir for 20
created dir for 21
created dir for 22
created dir for X
created dir for Y
created dir for MT


In [4]:
D = {}
with open(cfg,'r') as f:
    lines = [l.strip() for l in f.readlines() if l]
fn = lambda X: X.split('.')[-2]
for k,g in it.groupby(sorted(lines,key=fn),key=fn):
    D[k] = list(g)
D.keys()

dict_keys(['23andme', '23andme-exome-vcf', 'IYG', 'ancestry', 'decodeme', 'ftdna-illumina', 'genes-for-good'])

In [5]:
txt = ['23andme','23andme-exome-vcf','ancestry','genes-for-good']
gzd = ['IYG','decodeme','ftdna-illumina']

In [15]:
def read_23andme(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = '\t'
    pg = io.StringIO(pagebytes)
    args = {
        'comment':'#','sep':sep,'dtype':object,'names':cols
    }
    return pd.read_csv(pg,**args)
def read_ancestry(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = ','
    args = {
        'skiprows':1,'sep':sep,'dtype':object,'names':cols
    }
    filt = lambda Y: not Y.startswith('#')
    decd = lambda X: X.replace('\t',',',3).replace('\t','')
    junk = pagebytes
    fpg = filter(filt,junk.splitlines())
    junk = [decd(x) for x in fpg]
    junk = io.StringIO('\n'.join(junk))
    return pd.read_csv(junk,**args)

In [7]:
def link_grab(Links):

    reqs = []
    m = pycurl.CurlMulti()
    for url in Links:
        buff = io.BytesIO()
        handle = pycurl.Curl()
        handle.setopt(pycurl.URL, url)
        handle.setopt(pycurl.WRITEDATA, buff)
        req = (url, buff, handle)
        m.add_handle(req[2])
        reqs.append(req)
    # Perform multi-request, modified from:
    # https://github.com/rushtongarth/XnaPyCurl/blob/master/src/getters/multi.py
    num_processed=0
    num_urls = len(Links)
    while num_processed < num_urls:
        while 1:
            ret, num_handles = m.perform()
            if ret != pycurl.E_CALL_MULTI_PERFORM: break
            else: print(num_handles)
        while 1:
            num_q, ok_list, err_list = m.info_read()
            for c in ok_list:
                m.remove_handle(c)
                print("Success:",c.getinfo(pycurl.EFFECTIVE_URL))
            for c, errno, errmsg in err_list:
                m.remove_handle(c)
                print("Failed: ", c, errno, errmsg)
            num_processed = num_processed + len(ok_list) + len(err_list)
            if num_q == 0:  break
        m.select(1.0)
    return reqs


In [22]:
def file_proc(to_proc,mode='23andme',verbose=False):
    #C = pycurl.Curl()
    #C.setopt(pycurl.MAXREDIRS,50)
    #buff = io.BytesIO()
    f1 = lambda Y: Y and not Y.startswith('#')
    f2 = lambda X: '-'.join(X.split('/')[-1].split('.')[::2])
    rows = {}
    if mode=='23andme':
        to_df = read_23andme
    elif mode=='ancestry':
        to_df = read_ancestry

    for u,req,h in to_proc:
        try:
            to_dec = req.getvalue().decode('ascii')
            genes = to_df(to_dec)
            rows[f2(u)] = genes
        except UnicodeDecodeError as e:
            print('Skipping %s'%u)
    return rows

In [9]:
subset = len(D['23andme'])//100
print(subset)

tograb = np.random.choice(D['23andme'],subset)
genes = link_grab(tograb)

34
Success: https://www.opensnp.org/data/4803.23andme.3386
Success: https://www.opensnp.org/data/3066.23andme.1971
Success: https://www.opensnp.org/data/2927.23andme.1861
Success: https://www.opensnp.org/data/5237.23andme.4625
Success: https://www.opensnp.org/data/7251.23andme.5630
Success: https://www.opensnp.org/data/2647.23andme.1665
Success: https://www.opensnp.org/data/1176.23andme.612
Success: https://www.opensnp.org/data/4607.23andme.3209
Success: https://www.opensnp.org/data/1648.23andme.2932
Success: https://www.opensnp.org/data/5210.23andme.3720
Success: https://www.opensnp.org/data/3729.23andme.2467
Success: https://www.opensnp.org/data/2730.23andme.1737
Success: https://www.opensnp.org/data/3527.23andme.2300
Success: https://www.opensnp.org/data/1727.23andme.999
Success: https://www.opensnp.org/data/6968.23andme.5365
Success: https://www.opensnp.org/data/5385.23andme.3899
Success: https://www.opensnp.org/data/4365.23andme.2937
Success: https://www.opensnp.org/data/2208.23an

In [23]:
L = file_proc(genes)

Skipping https://www.opensnp.org/data/1648.23andme.2932
Skipping https://www.opensnp.org/data/3729.23andme.2467
Skipping https://www.opensnp.org/data/5237.23andme.4625


In [37]:

for k,fr in L.items():
    for chrm,g in fr.groupby('chrom'):
        if chrm.isdecimal():
            
len(s)

0

,rsid,chrom,pos,alleles
593481,i4000095,Y,2649694,T
593482,i4000093,Y,2649696,--
593483,rs11575897,Y,2655180,G
593484,i5004371,Y,2655248,G
593485,i5004367,Y,2655265,T
593486,i5004381,Y,2655308,C
593487,i5004386,Y,2655319,A
593488,i5004379,Y,2655324,C
593489,i5004380,Y,2655325,--
593490,i5004382,Y,2655328,T


In [44]:
k1,k2 = list(L.keys())[:2]
f1,f2 = L[k1],L[k2]
f1.chrom.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'Y', 'MT'], dtype=object)